# 🧠 Proyecto de final de módulo

En este proyecto final recopilaremos los conocimientos adquiridos durante este primer módulo y nos centraremos en las habilidades de los distintos roles implicados en la construcción y análisis de los conjuntos de datos. Dentro del ciclo de vida del dato nos hemos centrado en tres aspectos clave:

* 🔧 **Data Engineering**: Construir conjuntos de datos acorde a las necesidades de nuestro negocio. Empleando herramientas como Python y todas sus extensiones, webscrapping y consultas de APIs. Junto con SQL y ser capaz de modelar tablas en nuestro destino, podemos organizar la información para que no solo sea útil hoy sino en el futuro de nuestro negocio.
* 🔍 **Data Scientist**: Ser capaz de analizar los datos para entender su correcta construcción, población que representan nuestras muestras. Posibles problemas y sesgos implicados en la recolección de los datos, etc. Empleando similares herramientas pero enfocado en paquetes clave (Jupyter, Pandas, Numpy, Plotly) podemos crear evidencias que nos lleven a una mejor comprensión del contexto de los datos.
* 📊 **Data Analyst**: Ser capaz de condensar toda la información presente en nuestro sistema y presentársela a usuarios clave de forma que el mensaje sea claro y conciso, atractivo y lo menos sesgado posible. Para ello, las herramientas clave serán las plataformas de BI y las bases de datos que soportarán las operaciones requeridas por estas soluciones.

Para poner a prueba nuestras habilidades en estos roles tomaremos como punto de partida los datos accesibles en multitud de plataformas sobre las **elecciones europeas** . 

#### Nuestro empleador requiere que analicemos los datos obtenidos para entender el contexto político y las posibles tendencias que puedan surgir de cara a las próximas elecciones tanto europeas como locales.

In [3]:
# importamos las librerías necesarias:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import csv
from io import StringIO

Construiremos nuestra Base de Datos MySQL en el servidor de [Aiven](https://aiven.io/).

In [ ]:
# Conectamos con el servidor:

engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{db}")
con = engine.connect()

# Pese a que en su origen no aparezca, es necesario añadir el "+pymysql", y recomendable borrar los carácteres tras el nombre de la base de datos!

Y más adelante, tras el proceso de **Data Scientist**, crearemos y gestionaremos las tablas mediante la herramienta [DBeaver](https://dbeaver.com/) tras importarlas.  
  
La seccion final (**Data Analyst**) constara de cuatro versiones o análisis realizados en base a los cuales se realizarán distintos procesos de busqueda y limpieza de datos. Dichas fases serán las siguientes:  
+ **V1: Datos actuales generales y nacionales**
+ **V2: Comparación con el pasado**
+ **V3: Comparación económica (PIB per capita), top1 vs bottom1**
+ **V4: Comparación autonómica vs nacional**

## 🔧 **Data Engineering**  

Comencemos por recopilar datos para la primera versión o análisis. Para ello, usaremos como fuente de información, [la plataforma de la UE](https://results.elections.europa.eu/es/), la cual nos servirá también para obtener los datos necesarios para las segunda y tercera versiones o análisis.  

Estructura de los csv que descargar de dicha plataforma (sin relaciones aún), tras un primer vistazo:  
![tablas y contenido](./img/diagrama_es_2024.png)  
  
Todas las columnas de "update_status" y "update_time" son irrelevantes de cara a análisis, por lo que las eliminaremos durante el proceso de limpieza de datos (sección Data Scientist). Además, podrían venir bien un par de tablas extra, en las que especifiquemos los valores de los "languaje_id", "country_id" y "division_id", asignando a dichas columnas como claves foráneas.

Para la cuarta versión o análisis, es decir, la relacionada con los datos locales, usaremos los siguientes datos: [Datos de Euskadi](https://datos.gob.es/es/catalogo?q=elecciones+europeas&sort=score+desc%2C+metadata_created+desc), cuya estructura es muy desapropiada para un .csv común:  
    
![imagen csv euskadi](./img/eus.png)

## 🔍 **Data Scientist**
#### En la primera versión de la entrega, nos centraremos en un espectro general de la Union y en el nacional.

In [ ]:
groups = pd.read_csv("data/groups.csv", sep=";", index_col=False)
print(groups["ID"].unique().__len__())

10


In [ ]:
parlamento = pd.read_csv("data/eu.csv", sep=";", index_col=False)
print(parlamento["GROUP_ID"].unique().__len__())

9


Parece que existe un grupo o coalición sin asientos en el parlamento europeo. Por ello también crearemos una tabla de clave primaria para múltiples claves foráneas de otras tablas.

In [ ]:
grupos = groups["ID"].unique()
print(grupos)

['EPP' 'SD' 'ECR' 'Renew' 'Theleft' 'GREENSEFA' 'PfE' 'ESN' 'NI' 'Others']


Tenemos identificadas las coaliciones politicas.  
  
Vamos a por las regiones o divisiones europeas!

In [ ]:
parties = pd.read_csv("data/parties.csv", sep=";", index_col=False)
parties_copia = parties.copy()


In [ ]:
print(parties["DIVISION_ID"].unique().__len__()) 
# 30 divisiones/países pero sólo hay 27 Estados miembros de UE! A identificar los sobrantes y eliminarlos!

# FBE (European Banking Federation)
# GBE (Governing Body Endorsement)
# WBE (Dirección General de Presupuesto, o DG BUDG)

30


In [ ]:
valores_a_eliminar = ["FBE", "GBE", "WBE"]
parties_copia = parties_copia[~parties_copia["DIVISION_ID"].isin(valores_a_eliminar)]
print(parties_copia["DIVISION_ID"].unique().__len__())

27


In [ ]:
divisiones = parties_copia["DIVISION_ID"].unique()
print(divisiones)

['BE' 'BG' 'CZ' 'DK' 'DE' 'EE' 'IE' 'EL' 'ES' 'FR' 'IT' 'CY' 'LV' 'LT'
 'LU' 'HU' 'MT' 'NL' 'AT' 'PL' 'PT' 'RO' 'SI' 'SK' 'FI' 'SE' 'HR']


Ya tenemos la lista de Estados Miembro de la Unión Europea!
Desde [este enlace](https://vlex.es/vid/abreviaturas-oficiales-paises-union-591203078) conseguimos los nombres de los países de cada abreviatura, y de ahí crearemos una lista con la que poder completar nuestra tabla de divisiones (ID, NAME)

In [ ]:
nombres = ["Belgica", "Bulgaria", "Republica Checa", "Dinamarca", "Alemania", "Estonia", "Irlanda", "Grecia", "Espanya", "Francia", "Italia", "Chipre", "Letonia", "Lituania", "Luxemburgo", "Hungria", "Malta", "Paises Bajos", "Austria", "Polonia", "Portugal", "Rumania", "Eslovenia", "Eslovaquia", "Finlandia", "Suecia", "Croacia"]

In [ ]:
division = [{"ID": i, "NAME": n} for i, n in zip(divisiones, nombres)]
print(division)

[{'ID': 'BE', 'NAME': 'Belgica'}, {'ID': 'BG', 'NAME': 'Bulgaria'}, {'ID': 'CZ', 'NAME': 'Republica Checa'}, {'ID': 'DK', 'NAME': 'Dinamarca'}, {'ID': 'DE', 'NAME': 'Alemania'}, {'ID': 'EE', 'NAME': 'Estonia'}, {'ID': 'IE', 'NAME': 'Irlanda'}, {'ID': 'EL', 'NAME': 'Grecia'}, {'ID': 'ES', 'NAME': 'Espanya'}, {'ID': 'FR', 'NAME': 'Francia'}, {'ID': 'IT', 'NAME': 'Italia'}, {'ID': 'CY', 'NAME': 'Chipre'}, {'ID': 'LV', 'NAME': 'Letonia'}, {'ID': 'LT', 'NAME': 'Lituania'}, {'ID': 'LU', 'NAME': 'Luxemburgo'}, {'ID': 'HU', 'NAME': 'Hungria'}, {'ID': 'MT', 'NAME': 'Malta'}, {'ID': 'NL', 'NAME': 'Paises Bajos'}, {'ID': 'AT', 'NAME': 'Austria'}, {'ID': 'PL', 'NAME': 'Polonia'}, {'ID': 'PT', 'NAME': 'Portugal'}, {'ID': 'RO', 'NAME': 'Rumania'}, {'ID': 'SI', 'NAME': 'Eslovenia'}, {'ID': 'SK', 'NAME': 'Eslovaquia'}, {'ID': 'FI', 'NAME': 'Finlandia'}, {'ID': 'SE', 'NAME': 'Suecia'}, {'ID': 'HR', 'NAME': 'Croacia'}]


Al igual que con los países, para identificar los idiomas y sus abreviaturas necesitaremos información externa, concretamente de [este enlace](https://omegat.sourceforge.io/manual-latest/es/appendix.languages.html).

In [ ]:
language = pd.read_csv("data/labels.csv", sep=";")
print(language["LANGUAGE_ID"].unique().__len__()) # 24 idiomas

24


In [ ]:
idiomas = language["LANGUAGE_ID"].unique()
print(idiomas)

['BG' 'ES' 'CS' 'DA' 'DE' 'ET' 'EL' 'EN' 'FR' 'GA' 'HR' 'IT' 'LV' 'LT'
 'HU' 'MT' 'NL' 'PL' 'PT' 'RO' 'SK' 'SL' 'FI' 'SV']


In [ ]:
nom_idi = ["Bulgaro", "Espanyol", "Checo", "Danes", "Aleman", "Estonio", "Griego Moderno", "Ingles", "Frances", "Irlandes", "Croata", "Italiano", "Leton", "Lituano", "Hungaro", "Maltes", "Holandes", "Polaco", "Portugues", "Rumano", "Eslovaco", "Esloveno", "Finlandes", "Sueco"]

In [ ]:
language = [{"ID": i, "NAME": n} for i, n in zip(idiomas, nom_idi)]
print(language)

[{'ID': 'BG', 'NAME': 'Bulgaro'}, {'ID': 'ES', 'NAME': 'Espanyol'}, {'ID': 'CS', 'NAME': 'Checo'}, {'ID': 'DA', 'NAME': 'Danes'}, {'ID': 'DE', 'NAME': 'Aleman'}, {'ID': 'ET', 'NAME': 'Estonio'}, {'ID': 'EL', 'NAME': 'Griego Moderno'}, {'ID': 'EN', 'NAME': 'Ingles'}, {'ID': 'FR', 'NAME': 'Frances'}, {'ID': 'GA', 'NAME': 'Irlandes'}, {'ID': 'HR', 'NAME': 'Croata'}, {'ID': 'IT', 'NAME': 'Italiano'}, {'ID': 'LV', 'NAME': 'Leton'}, {'ID': 'LT', 'NAME': 'Lituano'}, {'ID': 'HU', 'NAME': 'Hungaro'}, {'ID': 'MT', 'NAME': 'Maltes'}, {'ID': 'NL', 'NAME': 'Holandes'}, {'ID': 'PL', 'NAME': 'Polaco'}, {'ID': 'PT', 'NAME': 'Portugues'}, {'ID': 'RO', 'NAME': 'Rumano'}, {'ID': 'SK', 'NAME': 'Eslovaco'}, {'ID': 'SL', 'NAME': 'Esloveno'}, {'ID': 'FI', 'NAME': 'Finlandes'}, {'ID': 'SV', 'NAME': 'Sueco'}]


Ya tenemos la lista de idiomas a los que se traducen los documentos en la Unión Europea!  
   
Ahora creemos nuestras nuevas tablas para tras todo el proceso de limpieza y ajuste de datos, poder importarlas al DBeaver.

In [ ]:
# Las divisiones/países de la Unión:

with open("./data_final/division.csv", "w", newline="") as csvfile:
    fieldnames = ["ID", "NAME"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    writer.writerows(division)

In [ ]:
# Las idiomas de la Unión:

with open("./data_final/language.csv", "w", newline="") as csvfile:
    fieldnames = ["ID", "NAME"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    writer.writerows(language)


Una vez creadas las que serán 3 de nuestras tablas base, vayamos adecuando las demás. Como dijimos en la sección anterior, existen en varias tablas (eu, turnout-eu, turnout-country, pais-result-parties, pais-groups y pais-parties) 2 columnas sin importancia para el análisis, por lo que nos desprenderemos de ellas en las próximas celdas.

In [ ]:
# tabla eu

df = pd.read_csv("./data/eu.csv", sep=";")

columnas_deseadas = ["GROUP_ID", "SEATS_TOTAL", "SEATS_PERCENT_EU"]
df_filtrado = df[columnas_deseadas]

df_filtrado.to_csv("./data_final/eu.csv", index=False)

In [ ]:
# tabla turnout-eu

df = pd.read_csv("./data/turnout-eu.csv", sep=";")

columnas_deseadas = ["YEAR", "RATE"]
df_filtrado = df[columnas_deseadas]

df_filtrado.to_csv("./data_final/turnout-eu.csv", index=False)

In [ ]:
# tabla turnout-country

df = pd.read_csv("./data/turnout-country.csv", sep=";")

columnas_deseadas = ["YEAR", "COUNTRY_ID", "RATE"]
df_filtrado = df[columnas_deseadas]

df_filtrado.to_csv("./data_final/turnout-country.csv")

In [ ]:
# tabla es-results-parties

df = pd.read_csv("./data/es-results-parties.csv", sep=";")

columnas_deseadas = ["PARTY_ID", "TYPE", "VOTES_PERCENT"]
df_filtrado = df[columnas_deseadas]

df_filtrado.to_csv("./data_final/es-results-parties.csv", index=False)

In [ ]:
# tabla es-parties

df = pd.read_csv("./data/es-parties.csv", sep=";")

columnas_deseadas = ["PARTY_ID", "TYPE", "SEATS_TOTAL", "GROUP_ID"]
df_filtrado = df[columnas_deseadas]

df_filtrado.to_csv("./data_final/es-parties.csv")

In [ ]:
# tabla es-groups

df = pd.read_csv("./data/es-groups.csv", sep=";")

columnas_deseadas = ["GROUP_ID", "SEATS_TOTAL", "SEATS_PERCENT_EU"]
df_filtrado = df[columnas_deseadas]

df_filtrado.to_csv("./data_final/es-groups.csv", index=False)

Tras este proceso de limpieza de columnas, nos quedan aún unas tablas (parties, groups) fuera de la carpeta "data_final".

In [ ]:
# tabla de parties

columnas_deseadas = ["DIVISION_ID", "ID", "TYPE", "ACRONYM", "LABEL", "MEMBER_OF_COALITION", "COALITION_MEMBERS"]
df_filtrado = parties_copia[columnas_deseadas]

df_filtrado.to_csv("./data_final/parties.csv", index=False)

Para poder realizar un análisis más profundo de las tendencias, tanto locales como internacionales, añadiremos dos columnas que detallen la orientación y la ideología de cada uno de los grupos políticos o coaliciones que forman parte del parlamanto de la Unión Europea.

In [ ]:
# tabla de groups

df_filtrado = groups[groups["LANGUAGE_ID"] == "ES"]

orientacion = [
    "Centro-derecha",
    "Centro-izquierda",
    "Derecha a derecha moderada",
    "Centro, liberal-progresista",
    "Izquierda a extrema izquierda",
    "Centro-izquierda / ecologista",
    "Extrema derecha / ultraconservadora",
    "Extrema derecha / ultranacionalista",
    "",
    ""
]

ideologia = [
    "Conservadurismo moderado, democracia cristiana, economía social de mercado, europeísmo",
    "Socialdemocracia, progresismo, estado del bienestar, europeísmo",
    "Conservadurismo euroescéptico, soberanismo, liberalismo económico",
    "Liberalismo, europeísmo, derechos civiles, economía de mercado",
    "Socialismo, comunismo democrático, anticapitalismo, ecologismo, eurocrítica",
    "Ecologismo, federalismo europeo, derechos humanos, progresismo",
    "Nacionalismo, soberanismo, antiinmigración, euroescepticismo radical",
    "Soberanismo duro, nacionalismo, anti-UE, identidad cultural",
    "",
    ""
]

df_filtrado["ORIENTACION"] = orientacion
df_filtrado["IDEOLOGIA"] = ideologia

df_filtrado.to_csv("./data_final/groups.csv", index=False)

C:\Users\iktet\AppData\Local\Temp\ipykernel_9164\373878701.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado["ORIENTACION"] = orientacion
C:\Users\iktet\AppData\Local\Temp\ipykernel_9164\373878701.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado["IDEOLOGIA"] = ideologia


Ahora ya sí, los ajustes y la limpieza han terminado. Nos quedaría una estructura como esta:  
![tablas relaciones y contenido](./img/diagrama_final.png) 

#### Y hasta aquí la limpieza de datos de la primera versión de entrega.
---
#### En la segunda versión, se agregarán tablas de pasadas elecciones (concretamente del año 1994, tras la unificación alemana) nacionales para comparar las tendencias pasadas con las actuales.

In [ ]:
# Empecemos con las regiones o divisiones:

parties_1994 = pd.read_csv("data/parties-1994.csv", sep=";", index_col=False)
parties_1994_copia = parties_1994.copy()

In [ ]:
print(parties_1994["DIVISION_ID"].unique().__len__())
# igual que en la versión anterior, nos encontramos con valores que no representan a paises, sino organizaciones

17


In [ ]:
divisiones_1994 = parties_1994_copia["DIVISION_ID"].unique()
print(divisiones_1994) # tres valores que no estaban en la lista de parties de 2024, habrá que crear una nueva tabla, o añadirlos a la ya existente

['BE' 'FBE' 'GBE' 'WBE' 'DK' 'DE' 'IE' 'EL' 'ES' 'FR' 'IT' 'LU' 'NL' 'PT'
 'UK' 'NI' 'GB']


In [ ]:
no_coincidentes = []

for elemento in divisiones_1994:
    if elemento not in divisiones:
        if elemento not in no_coincidentes:
            no_coincidentes.append(elemento)

print(no_coincidentes)

['FBE', 'GBE', 'WBE', 'UK', 'NI', 'GB']


In [ ]:
valores_a_eliminar = ["FBE", "GBE", "WBE", "UK"] 
# "UK" representa la union de los valores en "NI" y "GB", por lo que impediría la creacion de una clave primaria al repetirse los valores de la columna ID
parties_1994_copia = parties_1994_copia[~parties_1994_copia["DIVISION_ID"].isin(valores_a_eliminar)]
print(parties_1994_copia["DIVISION_ID"].unique().__len__())

13


In [ ]:
valores_a_eliminar = ["UK90"] # se repite sin necesidad, obstaculizando el proceso de creación de la tabla en base a clave primaria
parties_1994_copia = parties_1994_copia[~parties_1994_copia["ID"].isin(valores_a_eliminar)]

In [ ]:
with open("./data_final/division.csv", "a", newline="") as csvfile:
    writer = csv.writer(csvfile)

    datos1, datos2, datos3 = ["UK", "Reino Unido"], ["NI", "Irlanda del Norte"], ["GB", "Gran Bretanya"]

    writer.writerow(datos1)
    writer.writerow(datos2)
    writer.writerow(datos3)

Tabla de divisiones actualizada!  
  
Vamos con los partidos politicos nacionales.

In [ ]:
columnas_deseadas = ["DIVISION_ID", "ID", "TYPE", "ACRONYM", "LABEL", "MEMBER_OF_COALITION", "COALITION_MEMBERS"]
df_filtrado = parties_1994_copia[columnas_deseadas]

df_filtrado.to_csv("./data_final/parties-1994.csv", index=False)

Partidos nacionales limpiados!  
  
Vamos con los antiguos grupos politicos.

In [ ]:
groups_1994 = pd.read_csv("data/groups-1994.csv", sep=";", index_col=False, keep_default_na=False) # para evitar que borre "NA" al detectarlo como NaN


In [ ]:
# tabla de groups de 1994

df_filtrado = groups_1994[groups_1994["LANGUAGE_ID"] == "ES"]

df_filtrado.to_csv("./data_final/groups-1994.csv", index=False) 

Grupos o coaliciones terminadas!  
  
Vamos con el borrado de columnas que ya hicimos anteriormente en las tablas equivalentes de 2024.

In [ ]:
# tabla eu

df = pd.read_csv("./data/eu-1994.csv", sep=";", keep_default_na=False)

columnas_deseadas = ["GROUP_ID", "SEATS_TOTAL", "SEATS_PERCENT_EU"]
df_filtrado = df[columnas_deseadas]

df_filtrado.to_csv("./data_final/eu-1994.csv", index=False)

In [ ]:
# tabla es-groups-1994

df = pd.read_csv("./data/es-groups-1994.csv", sep=";", keep_default_na=False)

columnas_deseadas = ["GROUP_ID", "SEATS_TOTAL", "SEATS_PERCENT_EU"]
df_filtrado = df[columnas_deseadas]

df_filtrado.to_csv("./data_final/es-groups-1994.csv", index=False)

In [ ]:
# tabla es-parties-1994

df = pd.read_csv("./data/es-parties-1994.csv", sep=";", keep_default_na=False)

columnas_deseadas = ["PARTY_ID", "TYPE", "SEATS_TOTAL", "GROUP_ID"]
df_filtrado = df[columnas_deseadas]

df_filtrado.to_csv("./data_final/es-parties-1994.csv", index=False)

#### Y hasta aquí la limpieza de datos de la segunda versión de entrega.
---
#### En la tercera versión, se agregarán tablas de las últimas elecciones de otros países, para comparar sus posibles distintas tendencias en base a su situación económica. Más concretamente, se trabajará con las tablas del top1 y bottom1 del [ranking europeo de PIB per capita](https://datosmacro.expansion.com/paises/grupos/union-europea). En dicho ranking, vemos cómo España se situa ligeramente por debajo de la media, lo cual también podrá ser de utilidad para la comparativa.  
  
![top3_eu](./img/top3_pib_eu.png)
![bottom3_eu](./img/bot3_pib_eu.png)  
   
Empecemos con los partidos nacionales de Luxemburgo (LU) y Bulgaria (BG).

In [ ]:
# tabla lu-results-parties

df = pd.read_csv("./data/lu-results-parties.csv", sep=";")

columnas_deseadas = ["PARTY_ID", "TYPE", "VOTES_PERCENT"]
df_filtrado = df[columnas_deseadas]

df_filtrado.to_csv("./data_final/lu-results-parties.csv", index=False)

In [ ]:
# tabla lu-parties

df = pd.read_csv("./data/lu-parties.csv", sep=";")

columnas_deseadas = ["PARTY_ID", "TYPE", "SEATS_TOTAL", "GROUP_ID"]
df_filtrado = df[columnas_deseadas]

df_filtrado.to_csv("./data_final/lu-parties.csv", index=False)

In [ ]:
# tabla lu-groups

df = pd.read_csv("./data/lu-groups.csv", sep=";")

columnas_deseadas = ["GROUP_ID", "SEATS_TOTAL", "SEATS_PERCENT_EU"]
df_filtrado = df[columnas_deseadas]

df_filtrado.to_csv("./data_final/lu-groups.csv", index=False)

Luxemburgo terminado.  
  
A por las tablas de Bulgaria.

In [ ]:
# tabla bg-results-parties

df = pd.read_csv("./data/bg-results-parties.csv", sep=";")

columnas_deseadas = ["PARTY_ID", "TYPE", "VOTES_PERCENT"]
df_filtrado = df[columnas_deseadas]

df_filtrado.to_csv("./data_final/bg-results-parties.csv", index=False)

In [ ]:
# tabla bg-parties

df = pd.read_csv("./data/bg-parties.csv", sep=";")

columnas_deseadas = ["PARTY_ID", "TYPE", "SEATS_TOTAL", "GROUP_ID"]
df_filtrado = df[columnas_deseadas]

df_filtrado.to_csv("./data_final/bg-parties.csv", index=False)

In [ ]:
# tabla bg-groups

df = pd.read_csv("./data/bg-groups.csv", sep=";")

columnas_deseadas = ["GROUP_ID", "SEATS_TOTAL", "SEATS_PERCENT_EU"]
df_filtrado = df[columnas_deseadas]

df_filtrado.to_csv("./data_final/bg-groups.csv", index=False)

#### Y hasta aquí la limpieza de datos de la tercera versión de entrega.
---
#### En la cuarta y última versión, se agregarán tablas de las últimas elecciones de la comunidad autónoma de Euskadi, seccionada por sus 3 provincias, para comparar sus tendencias con las nacionales.

In [ ]:
with open("./data/xls0000192_c.csv", "r", encoding="latin1") as f:
    lines = f.readlines()

# Busquemos el índice donde comienzan realmente las filas de candidaturas
for i, line in enumerate(lines):
    if line.startswith("EH BILDU"):
        candidaturas_start = i
        break

# Extraemos las filas de candidaturas
candidature_rows = []
for line in lines[candidaturas_start:]:
    if line.strip() == "" or set(line.strip()) == {";"}:
        break
    candidature_rows.append(line.strip())

In [ ]:
eus = pd.read_csv(StringIO("\n".join(candidature_rows)), sep=";", header=None)

In [ ]:
# Asignamos nombres a nuestras columnas
eus.columns = [
    "Candidatura", 
    "Votos_Euskadi", "Percent_Euskadi", 
    "Votos_Araba", "Percent_Araba", 
    "Votos_Bizkaia", "Percent_Bizkaia", 
    "Votos_Gipuzkoa", "Percent_Gipuzkoa"
]

In [ ]:
# eliminamos comas y convertimos a float/int
for col in eus.columns[1:]:
    if col.startswith("Votos"):
        eus[col] = eus[col].astype(str).str.replace(".", "", regex=False).astype(int)
    else:
        eus[col] = eus[col].str.replace(",", ".", regex=False).astype(float)

In [ ]:
# Creamos los tres DataFrames por provincia
df_araba = eus[["Candidatura", "Votos_Araba", "Percent_Araba"]].copy()
df_bizkaia = eus[["Candidatura", "Votos_Bizkaia", "Percent_Bizkaia"]].copy()
df_gipuzkoa = eus[["Candidatura", "Votos_Gipuzkoa", "Percent_Gipuzkoa"]].copy()
df_eus = eus[["Candidatura", "Votos_Euskadi", "Percent_Euskadi"]].copy()

In [ ]:
df_eus.to_csv("./data_final/eus-eus.csv", index=False)
df_araba.to_csv("./data_final/eus-araba.csv", index=False)
df_bizkaia.to_csv("./data_final/eus-bizkaia.csv", index=False)
df_gipuzkoa.to_csv("./data_final/eus-gipuzkoa.csv", index=False)

Tras el proceso de limpieza y ajuste de las tablas implicadas en las cuatro versiones del análisis, el diagrama detallado de la base de datos final sería el siguiente:  
  
![diagrama absoluto](./img/diagrama_cuatro_versiones.png)  
  
Como podemos ver, consta de 3 secciones no relacionadas entre sí, ya sea por los diferentes integrantes políticos de cada época o entorno sociocultural.

## 📊 **Data Analyst**

En esta sección, utilizaremos [Preset](https://preset.io/), una herramienta que nos permite crear un entorno de desarrollo personalizado para nuestro proyecto.  
  
Comenzaremos con un "brain storming" de posibles gráficos o diseños a realizar, y luego procederemos a crear un entorno de desarrollo para implementarlos. En caso de no poder realizar alguno, lo adaptaremos en la medida de lo posible.

+ **Brain Storming**:
    - Semicirculo/Donutpie con coaliciones EU actuales
    - Gráfico participación por país (y EU total)... o mapa de calor?
    - Reparto de escaños nacionales en barra apilada... votos por partido?
    - ...
    - Gráfico de barras comparando participación nacional
    - Barra apilada escaños
    - ...
    - España cerca de valor medio europeo, añadir a comparación?
    - Gráficos barras comparativo escaños por grupo europeo y país
    - Tabla de partidos con votos y grupo europeo (por país)
    - ...
    - Barras apiladas comparativo escaños por grupo europeo y provincia/comunidad
  
  
+ **V1: Datos actuales generales y nacionales**
  
![v1](./img/v1.png)

+ **V2: Comparación con el pasado**

![v2](./img/v2.png)

+ **V3: Comparación económica (PIB per capita) internacional, top1 vs bottom1**

![v3](./img/v3.png)

+ **V4: Comparación autonómica vs nacional**

![v4](./img/v4.png)


## ✅ Conclusiones
  
+ **V1: Datos actuales generales y nacionales**
    - <u>Participación</u>: España, al igual que veremos más adelante con el PIB per capita, se situa ligeramente por debajo de la media europea (46.39% vs 50.74%).
    - <u>Orientación</u>: en ambos ámbitos, el partido o coalición que más escaños ha conseguido, se situa en el centro-derecha (EU: EPP, ES: PP), seguido por un centro-izquierda (EU: SD, ES: PSOE), dos de derecha extrema o derecha moderada (EU: Pfe y ECR, ES: VOX y SALF) y como quinta fuerza se situa la izquierda o centro-izquierda (EU: Renew,ES: Podemos).  
    - <u>Tendencia</u>: se nota gran tendencia al centro, al conformismo social, con mas vertientes hacia la derecha que izquierda. La comodidad económica o la sensación de la misma podrían justificar dicha tendencia.
+ **V2: Comparación con el pasado**
    - <u>Participación</u>: en España ha caído seriamente (entormo al 30%) el interés por participar en elecciones europeas desde hace 30 años, pese a la subida de las anteriores elecciones en 2019. Sin embargo, en Europa el descenso no ha sido tan notorio (10%).
    - <u>Orientación</u>: hace 3 décadas existían menos opciones de voto y por ello menos vertientes políticas estaban cubiertas, sin embargo, las 2 mayores fuerzas políticas (PP, PSOE) han mantenido sus puestos a lo largo del tiempo, seguidos antaño por la izquierda, en vez de por la extrema derecha como ahora.  
    Sin embargo, en Europa el centro-izquierda (PSE del 1994, SD del 2024) estaba posicionado por delante del actual líder (PPE del 1994, EPP del 2024); la tercera fuerza era también de la extrema derecha (ELDR de 1994, Pfe del 2024), pero la izquierda se hacia notar en el cuarto puesto (GUE) en vez de en el quinto de 2024 (Renew).
    - <u>Tendencia</u>: En España, la tendencia era más hacia la izquierda que hoy en día pese a dominar los del centro en todo momento. En Europa también se nota que la izquierda y centro-izquierda tenían mayor presencia que hoy en día. 
+ **V3: Comparación económica (PIB per capita) internacional, top1 vs bottom1**
    - <u>Participación</u>: parece ser que a mayor PIB per capita, mayor cercanía se siente por participar en relación a decisiones internacionales, ya que Luxemburgo (82.29%) casi hasta duplica a España (46.39%) y ni qué decir de Bulgaria (33.78%). 
    - <u>Orientación</u>: en los tres países, la coalición que más escaños ha recibido es la misma (EPP, centro-derecha).  
    Si bien en Luxemburgo no podemos realizar un análisis más profundo debido el límite de escaños que se le otorga (debido a la población), podemos observar que las demás coaliciones que obtienen escaño (SD, ECR, Renew, GREENSEFA), coinciden con el orden de la Unión Europea, salvo por la ausencia de la extrema derecha (Pfe).  
    Por parte de Bulgaria, lidera igualmente el centro-derecha (EPP), seguido muy de cerca en este caso de la quinta fuerza europea (Renew), orientada también al centro-derecha y después la extrema derecha (ESN), última en el rango europeo. El centro izquierda (SD) tiene presencia minoritaria con 2 respecto a los 15 que recibe la derecha en todas sus variantes.
    - <u>Tendencia</u>: pese a la clásica asociación en nuestras mentes de pobreza con izquierda revolucionaria, los datos nos muestran que a menor PIB per capita, mayor tendencia a la derecha, y a mayor PIB per capita, mayor tendencia al centro, o centro-izquierda. 
+ **V4: Comparación autonómica vs nacional**
    - <u>Participación</u>: pese a no reflejarse en el dashboard, tanto a nivel nacional como autonómico existen niveles similares de compromiso con Europa (ES: 46.39%, EUS: 50.8%). El dato autonómico está disponible en el archivo .csv del que hemos obtenido los datos referentes a los votos.
    - <u>Orientación</u>: en las elecciones autonómicas, el movimiento de izquierda e independentista (EH BILDU), supera a los dos grandes partidos nacionales (PSOE en segundo lugar y PP en cuarto), y entre ellos se colocaría el antaño independentista partido de derecha autonómico (PNV), haciendo ver que importan más a la sociedad los grupos locales que los nacionales. 
    Ambos partidos locales, consiguieron un escaño en el parlamento europeo; EH BILDU integrado en TheLeft (y la agrupación nacional "Ahora repúblicas"), séptima potencia europea (izquierda a izquierda extrema) y PNV integrado en Renew (y la agrupación nacional "CEUS"), quinta potencia europea (centro-derecha). 
    En las tres provincias de Euskadi, vemos una intensa pelea entre los ya mencionados líderes autonómicos, EH BILDU (lider en Gipuzkoa), PSOE (líder en Bizkaia y Araba), y PNV (segundo en Bizkaia y tercero en las demás). 
    - <u>Tendencia</u>: a nivel general en la comunidad autónoma de Euskadi se ve una clara tendencia a la izquierda (EH BILDU + PSOE = 52.45%), seguido por el centro-derecha (PNV + PP = 34.08%). 
    A nivel provincial, como ya hemos mencionado, la constante pelea entre la izquierda (EH BILDU) y derecha (PNV) local junto con el centro izquierda nacional (PSOE), es muy notoria pese a que el centro-derecha nacional (PP) también tenga fuerza en Araba (porcentaje doble que en Gipuzkoa).